In [1]:
import numpy as np # for printing matrix and vectors only for comparisons
from tabulate import tabulate # for printing matrix and vectors
from fractions import Fraction

## Question 1: Power Method and QR Decomposition

In [2]:
def read_matrix_vector(filename):
    """Read the matrix and vector from a text file. Able to handle fractions."""
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    matrix = []
    for line in lines[:]: 
        line = line.strip()
        row = []
        for val in line.split():
            # Handle fractions
            if '/' in val:
                numerator, denominator = val.split('/')
                frac_val = Fraction(int(numerator), int(denominator))
                row.append(float(frac_val))
            else:
                row.append(float(val))
        matrix.append(row)
    
    return np.array(matrix)

In [3]:
filename = 'q1_input.txt'
A = read_matrix_vector(filename)
    
print("Matrix:")
print(tabulate(A, tablefmt='fancy_grid'))

Matrix:
╒═══════════╤══════════╤══════════╤═════════╕
│  4        │ 0.666667 │ -1.33333 │ 1.33333 │
├───────────┼──────────┼──────────┼─────────┤
│  0.666667 │ 4        │  0       │ 0       │
├───────────┼──────────┼──────────┼─────────┤
│ -1.33333  │ 0        │  6       │ 2       │
├───────────┼──────────┼──────────┼─────────┤
│  1.33333  │ 0        │  2       │ 6       │
╘═══════════╧══════════╧══════════╧═════════╛


In [4]:
def largest_eigenvalue(matrix, initial_guess, tolerable_error=1e-6, max_iteration=1000):
    """Calculate the largest eigenvalue using the Power Method."""
    n = len(matrix)
    
    x = initial_guess
    
    lambda_old = 1.0
    
    # Power Method iteration
    for _ in range(max_iteration):
        # Multiply the matrix and the vector
        x = np.matmul(matrix, x)
        
        # new eigenvalue
        lambda_new = max(abs(x))
        
        x = x / lambda_new
        
        if abs(lambda_new - lambda_old) < tolerable_error:
            return lambda_new
        
        lambda_old = lambda_new
    
    # If max iterations reached without convergence, return None
    return None

In [5]:
initial_guess = np.ones(len(A))

# eigenvalues using the power method
largest_eigval_custom = largest_eigenvalue(A, initial_guess)

# eigenvalues using numpy's built-in function
eigenvalues = np.linalg.eigvals(A)
largest_eigval_np = max(abs(eigenvalues))

In [6]:
results = [
    ["Power Method", largest_eigval_custom],
    ["NumPy Function", largest_eigval_np]
]

print(tabulate(results, headers=["Method", "Largest Eigenvalue"], tablefmt="fancy_grid"))

╒════════════════╤══════════════════════╕
│ Method         │   Largest Eigenvalue │
╞════════════════╪══════════════════════╡
│ Power Method   │                    8 │
├────────────────┼──────────────────────┤
│ NumPy Function │                    8 │
╘════════════════╧══════════════════════╛


In [7]:
class QREigendecomposition:
    """
    QR eigenvalue algorithm for symmetric matrices
    """

    def __init__(self, store_intermediate=False):
        self.eigenvalues = None
        self.eigenvectors = None
        self.store_intermediate = store_intermediate

    def qr_factorization(self, X):
        """
        Compute the QR factorization of a square matrix using gram-schmidt 
        orthonormalization.
        """

        X = X.T # want first index to be the column index (for convenience)

        q, r = np.zeros(X.shape), np.zeros(X.shape) # preallocate
        q[0] = X[0] / np.linalg.norm(X[0])
        r[0] = X @ q[0]
        for i in range(1, X.shape[0]):
            q[i] = X[i] - np.sum(np.dot(X[i], q[:i].T) * q[:i].T, axis=-1)
            q[i] /= np.linalg.norm(q[i])

            ## Update the upper triangular matrix R
            r[i, i:] = X[i:] @ q[i]
        
        q = q.T # because we took transpose beforehand for easier indexing
        return q, r

    def find_eigensystem(self, X, max_iter=2000, tol=1e-6):
        """
        Eigenvalues and eigenvectors of a matrix
        """
        prev = np.copy(X)
        tq = np.identity(X.shape[0])
        if self.store_intermediate: self.intermediate = [np.copy(X)]
        for i in range(max_iter):
            q, r = self.qr_factorization(X)
            
            X = r @ q 

            tq = tq @ q # accumulate the eigenvector matrix

            if self.store_intermediate: self.intermediate.append(np.copy(X))

            ## Check for convergence and stop early if converged
            if np.linalg.norm(X - prev) < tol:
                break
            prev = np.copy(X)
        eigenvalues, eigenvectors = np.diag(X), tq
        sort_inds = np.argsort(eigenvalues)
        eigenvalues, eigenvectors = eigenvalues[sort_inds], eigenvectors[:, sort_inds]
        self.eigenvalues, self.eigenvectors = eigenvalues, eigenvectors
        return eigenvalues, eigenvectors

In [8]:
QRSolver = QREigendecomposition(store_intermediate=True)
eigenvalues, eigenvectors = QRSolver.find_eigensystem(A)

eigenvalues_list = eigenvalues.tolist()
eigenvectors_list = eigenvectors.tolist()

print("Eigenvalues:")
print(tabulate([["Eigenvalues"] + eigenvalues_list], tablefmt="fancy_grid"))
print("\nEigenvectors:")
print(tabulate(eigenvectors_list, headers=["Eigenvector " + str(i+1) for i in range(len(eigenvectors_list))], tablefmt="fancy_grid"))

Eigenvalues:
╒═════════════╤═══╤═══╤═══╤═══╕
│ Eigenvalues │ 2 │ 4 │ 6 │ 8 │
╘═════════════╧═══╧═══╧═══╧═══╛

Eigenvectors:
╒═════════════════╤═════════════════╤═════════════════╤═════════════════╕
│   Eigenvector 1 │   Eigenvector 2 │   Eigenvector 3 │   Eigenvector 4 │
╞═════════════════╪═════════════════╪═════════════════╪═════════════════╡
│       -0.707107 │    -1.68587e-07 │        0.707107 │     1.84787e-13 │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│        0.235702 │     0.942809    │        0.235702 │     5.34017e-11 │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│       -0.471405 │     0.235702    │       -0.471405 │     0.707107    │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│        0.471405 │    -0.235702    │        0.471405 │     0.707107    │
╘═════════════════╧═════════════════╧═════════════════╧═════════════════╛


In [9]:
eigenvalues_np, eigenvectors_np = np.linalg.eigh(A)

eigenvalues_np_list = eigenvalues_np.tolist()
eigenvectors_np_list = eigenvectors_np.tolist()

# Print eigenvalues and eigenvectors using tabulate
print("\nEigenvalues (NumPy):")
print(tabulate([["Eigenvalues"] + eigenvalues_np_list], tablefmt="fancy_grid"))
print("\nEigenvectors (NumPy):")
print(tabulate(eigenvectors_np_list, headers=["Eigenvector " + str(i+1) for i in range(len(eigenvectors_np_list))], tablefmt="fancy_grid"))



Eigenvalues (NumPy):
╒═════════════╤═══╤═══╤═══╤═══╕
│ Eigenvalues │ 2 │ 4 │ 6 │ 8 │
╘═════════════╧═══╧═══╧═══╧═══╛

Eigenvectors (NumPy):
╒═════════════════╤═════════════════╤═════════════════╤═════════════════╕
│   Eigenvector 1 │   Eigenvector 2 │   Eigenvector 3 │   Eigenvector 4 │
╞═════════════════╪═════════════════╪═════════════════╪═════════════════╡
│        0.707107 │    -6.28037e-16 │        0.707107 │      0          │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│       -0.235702 │     0.942809    │        0.235702 │     -1.4803e-16 │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│        0.471405 │     0.235702    │       -0.471405 │      0.707107   │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┤
│       -0.471405 │    -0.235702    │        0.471405 │      0.707107   │
╘═════════════════╧═════════════════╧═════════════════╧═════════════════╛


## Q2: Cubic Fitting with functional basis

In [10]:
def read_data(filename):
    """Read input data from a text file."""
    with open(filename, 'r') as file:
        data = [list(map(float, line.strip().split())) for line in file]
    return np.array(data)

def linear_regression(X, y):
    """Perform linear regression."""
    # Solve normal equations: (X^T*X)*beta = X^T*y
    beta = np.linalg.inv(X.T @ X) @ X.T @ y
    
    return beta.flatten()  # Flatten beta to get coefficients

In [11]:
def chebyshev_basis(x, order):
    """Compute Chebyshev polynomial basis up to the desired order."""
    basis = [np.ones_like(x), x]
    for i in range(2, order + 1):
        basis.append(2 * x * basis[-1] - basis[-2])
    return np.column_stack(basis)

def cubic_fit_with_chebyshev_basis(data, order=3):
    """Perform cubic fit using Chebyshev polynomial basis."""
    x = data[:, 0] 
    y = data[:, 1] 
    
    # Chebyshev polynomial basis
    X = chebyshev_basis(x, order)
    
    # linear regression
    coefficients = linear_regression(X, y)
    
    return coefficients

In [12]:
filename = 'q2_input.txt'  # Change this to your input file
data = read_data(filename)

coefficients_chebyshev = cubic_fit_with_chebyshev_basis(data)
coefficients_numpy = np.polyfit(data[:,0], data[:,1], 3)

In [13]:
# comparison using tabulate
comparison = [["Intercept", coefficients_chebyshev[0], coefficients_numpy[3], "a_o - a_2", coefficients_chebyshev[0] - coefficients_chebyshev[2]],
               ["Coefficient of x", coefficients_chebyshev[1], coefficients_numpy[2], "a_1 - 3a_3", coefficients_chebyshev[1] - 3*coefficients_chebyshev[3]],
               ["Coefficient of x^2", coefficients_chebyshev[2], coefficients_numpy[1], "2a_2", 2*coefficients_chebyshev[2]],
               ["Coefficient of x^3", coefficients_chebyshev[3], coefficients_numpy[0], "4a_3", 4*coefficients_chebyshev[3]]]
print("\nComparison of coefficients:")
print(tabulate(comparison, headers=["Coefficient", "Chebyshev", "Polynomial (NumPy)", "Chebyshev to Polynomial", "Polynomial (Made to Match)"], tablefmt="fancy_grid"))


Comparison of coefficients:
╒════════════════════╤═════════════╤══════════════════════╤═══════════════════════════╤══════════════════════════════╕
│ Coefficient        │   Chebyshev │   Polynomial (NumPy) │ Chebyshev to Polynomial   │   Polynomial (Made to Match) │
╞════════════════════╪═════════════╪══════════════════════╪═══════════════════════════╪══════════════════════════════╡
│ Intercept          │    -4.98945 │             0.574659 │ a_o - a_2                 │                     0.574659 │
├────────────────────┼─────────────┼──────────────────────┼───────────────────────────┼──────────────────────────────┤
│ Coefficient of x   │    10.4774  │             4.72586  │ a_1 - 3a_3                │                     4.72586  │
├────────────────────┼─────────────┼──────────────────────┼───────────────────────────┼──────────────────────────────┤
│ Coefficient of x^2 │    -5.56411 │           -11.1282   │ 2a_2                      │                   -11.1282   │
├──────────────────

In [14]:
def modified_basis(x):
    """Modify the basis functions for cubic fit."""
    return np.column_stack((np.ones_like(x), 2*x - 1, 8*x**2 - 8*x + 1, 32*x**3 - 48*x**2 + 18*x - 1))  # Basis for cubic fit

def cubic_fit_with_modified_basis(data):
    """Perform cubic fit using modified basis."""
    x = data[:, 0]  # Assuming first column is x values
    y = data[:, 1]  # Assuming second column is y values
    
    # Compute modified basis
    X = modified_basis(x)
    
    # Perform linear regression
    coefficients = linear_regression(X, y)
    
    return coefficients

In [16]:
# Perform cubic fit using modified basis
coefficients_modified_basis = cubic_fit_with_modified_basis(data)
coefficients_numpy = np.polyfit(data[:,0], data[:,1], 3)

# comparison using tabulate
comparison = [["Intercept", coefficients_modified_basis[0], coefficients_numpy[3], "a_o - a_1 + a_2 - a_4", coefficients_modified_basis[0] + coefficients_modified_basis[2] - coefficients_modified_basis[1] -coefficients_modified_basis[3]],
               ["Coefficient of x", coefficients_modified_basis[1], coefficients_numpy[2], "2a_1 - 8a_2 + 18a_3", 2*coefficients_modified_basis[1] + 18*coefficients_modified_basis[3] - 8*coefficients_modified_basis[2]],
               ["Coefficient of x^2", coefficients_modified_basis[2], coefficients_numpy[1], "8a_2 - 48a_3", 8*coefficients_modified_basis[2] - 48*coefficients_modified_basis[3]],
               ["Coefficient of x^3", coefficients_modified_basis[3], coefficients_numpy[0], "32a_3", 32*coefficients_modified_basis[3]]]
print("\nComparison of coefficients:")
print(tabulate(comparison, headers=["Coefficient", "Modified Chebyshev", "Polynomial (NumPy)", "Chebyshev to Polynomial", "Polynomial (Made to Match)"], tablefmt="fancy_grid"))


Comparison of coefficients:
╒════════════════════╤══════════════════════╤══════════════════════╤═══════════════════════════╤══════════════════════════════╕
│ Coefficient        │   Modified Chebyshev │   Polynomial (NumPy) │ Chebyshev to Polynomial   │   Polynomial (Made to Match) │
╞════════════════════╪══════════════════════╪══════════════════════╪═══════════════════════════╪══════════════════════════════╡
│ Intercept          │            1.16097   │             0.574659 │ a_o - a_1 + a_2 - a_4     │                     0.574659 │
├────────────────────┼──────────────────────┼──────────────────────┼───────────────────────────┼──────────────────────────────┤
│ Coefficient of x   │            0.393514  │             4.72586  │ 2a_1 - 8a_2 + 18a_3       │                     4.72586  │
├────────────────────┼──────────────────────┼──────────────────────┼───────────────────────────┼──────────────────────────────┤
│ Coefficient of x^2 │            0.0468498 │           -11.1282   │ 8a_2 -

The condition number of a matrix is a measure of how sensitive the solution of a linear system is to changes in the matrix or the right-hand side vector. It gives an indication of the stability of the solution with respect to small perturbations in the input data. There are different ways to calculate the condition number of a matrix, and each method has its relative merits depending on factors such as computational efficiency, accuracy, and the specific properties of the matrix being analyzed. Let's compare some common methods for calculating the condition number:

1. **Direct computation:** This method calculates the condition number using the formula $ \text{cond}(A) = \| A \| \cdot \| A^{-1} \| $. It provides accurate results but can be computationally expensive, especially for large matrices, due to the need to compute the matrix inverse.

2. **Singular value decomposition (SVD):** The condition number can be computed using the singular values of the matrix \( A \) obtained from its SVD. This method is more computationally efficient than direct computation and gives a good approximation of the condition number.

3. **Iterative methods:** Some iterative methods can estimate the condition number without explicitly computing the matrix inverse or SVD. While these methods may offer faster computations, they may sacrifice accuracy compared to direct or SVD-based methods.

